# Brain Mesh

## Import of  modules

In [1]:
import os.path as path

import mne
from mne import read_source_estimate

import numpy as np

In [2]:
import ipysurfer

import ipyvolume
ipyvolume.__version__

'0.4.5'

## Data reading

In [3]:
data_path = mne.datasets.sample.data_path()

subject = 'sample'
subjects_dir = path.join(data_path, 'subjects')
act_data = path.join(data_path, 'MEG/sample/sample_audvis-meg-eeg')

stc = read_source_estimate(act_data)

## Plot source estimates using given initial time

In [4]:
fig = ipysurfer.plot_source_estimates(stc, subject=subject, surface='inflated', hemi='split',
                          colormap='hot', smoothing_steps=10, foreground='red', views=['lat', 'med'],
                          transparent=False, alpha=1.0, time_viewer=False, subjects_dir=subjects_dir,
                          clim='auto', cortex='classic', size=300,
                          background="black", initial_time=0.09)

## Plot source estimates with time viewer and color bar

In [5]:
fig = ipysurfer.plot_source_estimates(stc, subject=subject, surface='inflated', hemi='both',
                          colormap='hot', smoothing_steps=10, time_unit='s', time_label='auto',
                          transparent=False, alpha=1.0, time_viewer=True, subjects_dir=subjects_dir,
                          clim='auto', size=300, background="black", initial_time=0.09, colorbar=True)

## Plot signed data

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse

# Process MEG data

data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'

raw = mne.io.read_raw_fif(raw_fname)  # already has an average reference
events = mne.find_events(raw, stim_channel='STI 014')

event_id = dict(aud_r=1)  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
raw.info['bads'] = ['MEG 2443', 'EEG 053']
picks = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                       exclude='bads')
baseline = (None, 0)  # means from the first instant to t = 0
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, reject=reject)

###############################################################################
# Compute regularized noise covariance
# ------------------------------------
#
# For more details see :ref:`tut_compute_covariance`.

noise_cov = mne.compute_covariance(
    epochs, tmax=0., method=['shrunk', 'empirical'], verbose=True)

###############################################################################
# Compute the evoked response
# ---------------------------
# Let's just use MEG channels for simplicity.

evoked = epochs.average().pick_types(meg=True)
del epochs  # to save memory

###############################################################################
# Inverse modeling: MNE/dSPM on evoked and raw data
# -------------------------------------------------

# Read the forward solution and compute the inverse operator
fname_fwd = data_path + '/MEG/sample/sample_audvis-meg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fname_fwd)

# make an MEG inverse operator
info = evoked.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)
del fwd

# Compute inverse solution
# ------------------------

method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2,
                    method=method, pick_ori='normal')

Opening raw data file /home/okozyn/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0
319 events found
Event IDs: [ 1  2  3  4  5 32]
72 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 3)
4 projection items activated
Loading data for 72 events and 106 original time points ...
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EOG : ['EOG 061']
    Reje

In [8]:
fig = ipysurfer.plot_source_estimates(stc, subject=subject, surface='inflated', hemi='split',
                          colormap='auto', smoothing_steps=10, time_unit='s', time_label='auto', views=['lat', 'med'],
                          transparent=False, alpha=1.0, time_viewer=True, subjects_dir=subjects_dir,
                          clim='auto', size=400, background="black", initial_time=0.09, colorbar=True)